In [23]:
# Importation des librairies spécifiques à l'enregistrement du fichier final : 
import os
import json
from datetime import datetime
import sys

In [24]:
# Importation des librairies classiques :
import pandas as pd

In [25]:
# Importation de la configuration des chemins : 
sys.path.append(os.path.abspath(os.path.join("..")))
import config

In [26]:
# Chemin vers le fichier metadata.json depuis le dossier initial : 
metadata_path = os.path.join(config.METADATA_DIR, "metadata.json")

# Lecture du fichier metadata :
with open(metadata_path, "r") as f:
    metadata = json.load(f)

# Récupération du chemin du fichier raw tel qu'enregistré dans metadata : 
raw_data_path = metadata.get("raw_data")

# Chargement du fichier CSV :
df = pd.read_csv(raw_data_path)

In [27]:
# Affichage des premières lignes :
display(df.head())

,Year,Mk,Cn,M (kg),Ewltp (g/km),Ft,Ec (cm3),Ep (KW),Erwltp (g/km),Fc
0,2021,ALFA ROMEO,GIULIA,1504,178,PETROL,1995,148,1.2,7.9
1,2021,ALFA ROMEO,GIULIA,1504,179,PETROL,1995,148,1.2,7.9
2,2021,ALFA ROMEO,GIULIA,1504,180,PETROL,1995,148,1.2,8.0
3,2021,ALFA ROMEO,GIULIA,1520,159,PETROL,1995,148,0.8,7.0
4,2021,ALFA ROMEO,GIULIA,1520,160,PETROL,1995,148,0.8,7.0


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300662 entries, 0 to 300661
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Year           300662 non-null  int64  
 1   Mk             300662 non-null  object 
 2   Cn             300657 non-null  object 
 3   M (kg)         300662 non-null  int64  
 4   Ewltp (g/km)   300662 non-null  int64  
 5   Ft             300662 non-null  object 
 6   Ec (cm3)       300662 non-null  int64  
 7   Ep (KW)        300662 non-null  int64  
 8   Erwltp (g/km)  300662 non-null  float64
 9   Fc             300662 non-null  float64
dtypes: float64(2), int64(5), object(3)
memory usage: 22.9+ MB


In [29]:
# Suppression des doublons potentiels à travers les années. 
# On ne prend pas en compte la colonne "Cn" car de nombreuses variations d'orthographe existent pour un même modèle. 

subset_cols = [col for col in df.columns if col not in ['Cn', 'Year']]
df = df.drop_duplicates(subset=subset_cols)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 251399 entries, 0 to 300661
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Year           251399 non-null  int64  
 1   Mk             251399 non-null  object 
 2   Cn             251394 non-null  object 
 3   M (kg)         251399 non-null  int64  
 4   Ewltp (g/km)   251399 non-null  int64  
 5   Ft             251399 non-null  object 
 6   Ec (cm3)       251399 non-null  int64  
 7   Ep (KW)        251399 non-null  int64  
 8   Erwltp (g/km)  251399 non-null  float64
 9   Fc             251399 non-null  float64
dtypes: float64(2), int64(5), object(3)
memory usage: 21.1+ MB


In [30]:
# Vérification de la colonne "Ft"

print(df['Ft'].unique())

# Travail de catégorisation nécessaire
# Passage en minuscules des catégories en doublon

df['Ft'] = df['Ft'].str.lower()
print(df['Ft'].unique())

# Compter les valeurs marquées comme "unknown"

count_unknown = df[df['Ft'] == 'unknown'].shape[0]
print(count_unknown)

# Affichage des quelques lignes concernées

print(df[df['Ft'] == 'unknown'])

# Suppression de ces lignes (majoritairement composées de NaN)

df = df[df['Ft'] != 'unknown']

['PETROL' 'DIESEL' 'PETROL/ELECTRIC' 'DIESEL/ELECTRIC' 'NG-BIOMETHANE'
 'LPG' 'E85' 'UNKNOWN' 'NG' 'petrol' 'diesel' 'petrol/electric'
 'diesel/electric' 'lpg' 'e85' 'ng']
['petrol' 'diesel' 'petrol/electric' 'diesel/electric' 'ng-biomethane'
 'lpg' 'e85' 'unknown' 'ng']
4
        Year       Mk                 Cn  M (kg)  Ewltp (g/km)       Ft  \
51161   2021  RENAULT               CLIO    1201           130  unknown   
51850   2021  RENAULT             KADJAR    1447           140  unknown   
81134   2021    VOLVO  V90 Cross Country    1982           170  unknown   
181119  2022   TOYOTA        TOYOTA C-HR    1495           110  unknown   

        Ec (cm3)  Ep (KW)  Erwltp (g/km)   Fc  
51161        999       67            1.9  5.8  
51850       1332      103            1.2  6.2  
81134       1969      145            1.2  6.5  
181119      1798       72            0.9  3.8  


In [31]:
# Rassemblement des variables
# NB : Le dictionnaire peut être complété en cas de valeurs différentes dans le dataset utilisé

dico_fuel = {'petrol': 'Essence',
             'hydrogen' : 'Essence',
             'e85': 'Essence',
             'lpg': 'Essence',
             'ng': 'Essence',
             'ng-biomethane' : 'Essence',
             'diesel': 'Diesel',
             'petrol/electric': 'Hybride',
             'diesel/electric': 'Hybride',
             'electric' : 'Electrique'
}

df['Ft'] = df['Ft'].replace(dico_fuel)

In [32]:
# Mise de côté des modèles électriques (qui n'émettent pas directement de CO2)

df = df[df['Ft'] != 'Electrique']

In [33]:
# Vérifications des valeurs uniques de la colonne "Mk"

df['Mk'].unique()

array(['ALFA ROMEO', 'ALPINE', 'AUDI', 'Audi',
       'AUTOMOBILI LAMBORGHINI S.P.A.', 'BENTLEY', 'Bentley', 'BMW',
       'CITROEN', 'CUPRA', 'DACIA', 'DS', 'DS AUTOMOBILES', 'FIAT',
       'FORD', 'FORD-CNG-TECHNIK', 'Ford-CNG-Technik', 'HONDA', 'Honda',
       'HYUNDAI', 'JAGUAR', 'JEEP', 'KIA', 'LAMBORGHINI', 'LAND ROVER',
       'LEXUS', 'MASERATI', 'MAZDA', 'Mazda', 'MERCEDES', 'MERCEDES BENZ',
       'MERCEDES-BENZ', 'Mercedes-Benz', 'MINI', 'MITSUBISHI', 'NISSAN',
       'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT', 'SEAT', 'SKODA', 'ŠKODA',
       'SOCIETE DES AUTOMOBILES ALPINE(SAA)', 'SUBARU', 'SUZUKI',
       'TOYOTA', 'TRIPOD', 'VOLKSWAGEN', 'VOLKSWAGEN  VW',
       'VOLKSWAGEN VW', 'VOLKSWAGEN, VW', 'VOLVO', 'VW', '?KODA',
       'CASELANI', 'FORD - CNG-TECHNIK', 'FORD (D)', 'FORD (USA)',
       'FORD-CNG TECHNIK', 'LANCIA', 'MAN', 'MOTO STAR', 'NILSSON',
       'NISSAN AUTOMOTIVE EUROPE', 'RENAULT TECH', 'RENAULT/CARPOL',
       'VOLKSWAGEN. VW', 'WAVECAMPER', 'ALLIED VEHICLE

In [34]:
# Passage en majuscules : 
df['Mk'] = df['Mk'].str.upper()

# Liste des marques les plus répandues en Europe : 
target_brands = ['CITROEN', 'FORD', 'FIAT', 'RENAULT', 'MERCEDES', 'BMW', 'VOLKSWAGEN', 'ALPINE', 
                 'INEOS', 'LAMBORGHINI', 'TOYOTA', 'JAGUAR', 'GREAT WALL MOTOR', 'CATERHAM', 'PEUGEOT', 
                 'MAN', 'OPEL', 'ALLIED VEHICLES', 'IVECO', 'MITSUBISHI', 'DS', 'MAZDA', 'SUZUKI', 
                 'SUBARU', 'HYUNDAI', "AUDI", "NISSAN", "SKODA", "SEAT", "DACIA", "VOLVO", "KIA",
                 "LAND ROVER", "MINI", "PORSCHE", "ALFA ROMEO", "SMART", "LANCIA", "JEEP"
                 ]

# Fonction pour extraire les marques connues des chaînes de caractères : 
def extract_brand(value):
    for brand in target_brands:
        if brand in value:
            return brand
    return value
df['Mk'] = df['Mk'].apply(extract_brand)

In [35]:
# Correction des fautes de frappe : 
dico_marque = {
    'DS': 'CITROEN',
    'VW': 'VOLKSWAGEN',
    '?KODA': 'SKODA',
    'ŠKODA': 'SKODA',
    'PSA AUTOMOBILES SA': 'PEUGEOT',
    'FCA ITALY': 'FIAT',
    'ALFA  ROMEO': 'ALFA ROMEO',
    'LANDROVER': 'LAND ROVER'
}
df['Mk'] = df['Mk'].replace(dico_marque)

# Suppression des marques trop peu connues : 

brands_to_delete = ['TRIPOD', 'API CZ', 'MOTO STAR', 'REMOLQUES RAMIREZ', 'AIR-BRAKES', 
                    'SIN MARCA', 'WAVECAMPER', 'CASELANI', 'PANDA']
df = df[~df['Mk'].isin(brands_to_delete)]
print(df[df['Mk'].isin(brands_to_delete)])

Empty DataFrame
Columns: [Year, Mk, Cn, M (kg), Ewltp (g/km), Ft, Ec (cm3), Ep (KW), Erwltp (g/km), Fc]
Index: []


In [36]:
# Suppression des occurences trop faibles : 

def filter_brands(df, col='Mk', threshold=5):
    brands = df[col].tolist()
    unique_brands = df[col].unique().tolist()
    filtered_brands = [brand for brand in unique_brands if brands.count(brand) >= threshold]
    return filtered_brands

filtered_brands = filter_brands(df, col='Mk', threshold=5)
df = df[df['Mk'].isin(filtered_brands)]

In [37]:
# Création d'une fonction pour détecter les valeurs aberrantes dans chaque colonne :

def detect_outliers(df, target_col, group_cols=["Cn", "Ft", "Year"]):
    # Calcul de la moyenne par groupe :
    stats = (
        df.groupby(group_cols)
          .agg(**{f'{target_col}_mean': (target_col, 'mean')})
          .reset_index()
    )
    
    # Fusion du DataFrame initial avec les statistiques calculées :
    df_merged = pd.merge(df, stats, on=group_cols, how="left")
    
    # Calcul de l'écart absolu entre la valeur et la moyenne :
    diff_col = f"diff_{target_col}"
    df_merged[diff_col] = (df_merged[target_col] - df_merged[f"{target_col}_mean"]).abs()
    
    # Calcul des quartiles et de l'IQR :
    q1 = df_merged[diff_col].quantile(0.25)
    q3 = df_merged[diff_col].quantile(0.75)
    iqr = q3 - q1
    
    # Calcul du seuil (Q3 + 1.5 * IQR) :
    seuil = (q3 + 1.5 * iqr).round(1)

    # Affichage du nombre d'outliers :
    nb_outliers = len(df_merged[df_merged[diff_col] >= seuil])
    print(f'Nombre de lignes dont la valeur de "{target_col}" dépasse le seuil de {seuil}: {nb_outliers}')
    
    # Suppression des lignes présentant des outliers :
    df_clean_no_outliers = df_merged[df_merged[diff_col] <= seuil]
    print(f"Nombre de lignes après suppression des outliers : {len(df_clean_no_outliers)}")
    
    return df_clean_no_outliers

In [38]:
# Liste des colonnes à filtrer successivement :
columns_to_filter = ['Ewltp (g/km)', 'Fc', 'M (kg)', 'Ec (cm3)', 'Ep (KW)', 'Erwltp (g/km)']

# On part du DataFrame initial (copie pour ne pas altérer l'original) :
df_temp = df.copy()

# Boucle sur chaque colonne pour appliquer le filtrage successif des outliers :
for col in columns_to_filter:
    print(col)
    df_temp = detect_outliers(df_temp, col)

print("\nAprès filtrage successif, le nombre de lignes restantes est de :", len(df_temp))

# Suppression des valeurs aberrantes après traitement :
df_clean_no_outliers_final = df_temp
display(df_clean_no_outliers_final.head())

# Suppression des colonnes ajoutées pour la détection de valeurs aberrantes afin d'éviter tout risque de fuite de données :
df_clean_no_outliers_final = df_clean_no_outliers_final[['Mk', 'Cn', 'M (kg)', 'Ewltp (g/km)', 'Ft', 'Ec (cm3)', 
                                                         'Ep (KW)', 'Erwltp (g/km)', 'Year', 'Fc']]

Ewltp (g/km)
Nombre de lignes dont la valeur de "Ewltp (g/km)" dépasse le seuil de 22.9: 11721
Nombre de lignes après suppression des outliers : 239663
Fc
Nombre de lignes dont la valeur de "Fc" dépasse le seuil de 1.1: 9417
Nombre de lignes après suppression des outliers : 230246
M (kg)
Nombre de lignes dont la valeur de "M (kg)" dépasse le seuil de 118.6: 9288
Nombre de lignes après suppression des outliers : 220958
Ec (cm3)
Nombre de lignes dont la valeur de "Ec (cm3)" dépasse le seuil de 104.7: 38061
Nombre de lignes après suppression des outliers : 182897
Ep (KW)
Nombre de lignes dont la valeur de "Ep (KW)" dépasse le seuil de 29.8: 5403
Nombre de lignes après suppression des outliers : 177494
Erwltp (g/km)
Nombre de lignes dont la valeur de "Erwltp (g/km)" dépasse le seuil de 0.8: 3685
Nombre de lignes après suppression des outliers : 173831

Après filtrage successif, le nombre de lignes restantes est de : 173831


,Year,Mk,Cn,M (kg),Ewltp (g/km),Ft,Ec (cm3),Ep (KW),Erwltp (g/km),Fc,...,Fc_mean,diff_Fc,M (kg)_mean,diff_M (kg),Ec (cm3)_mean,diff_Ec (cm3),Ep (KW)_mean,diff_Ep (KW),Erwltp (g/km)_mean,diff_Erwltp (g/km)
0,2021,ALFA ROMEO,GIULIA,1504,178,Essence,1995,148,1.2,7.9,...,7.225862,0.674138,1545.034483,41.034483,1995.0,0.0,173.0,25.0,0.836364,0.363636
1,2021,ALFA ROMEO,GIULIA,1504,179,Essence,1995,148,1.2,7.9,...,7.225862,0.674138,1545.034483,41.034483,1995.0,0.0,173.0,25.0,0.836364,0.363636
2,2021,ALFA ROMEO,GIULIA,1504,180,Essence,1995,148,1.2,8.0,...,7.225862,0.774138,1545.034483,41.034483,1995.0,0.0,173.0,25.0,0.836364,0.363636
3,2021,ALFA ROMEO,GIULIA,1520,159,Essence,1995,148,0.8,7.0,...,7.225862,0.225862,1545.034483,25.034483,1995.0,0.0,173.0,25.0,0.836364,0.036364
4,2021,ALFA ROMEO,GIULIA,1520,160,Essence,1995,148,0.8,7.0,...,7.225862,0.225862,1545.034483,25.034483,1995.0,0.0,173.0,25.0,0.836364,0.036364


In [39]:
# Mise de côté des modèles hybrides trop peu représentés : 
df_clean_no_outliers_final = df_clean_no_outliers_final[df_clean_no_outliers_final['Ft'] != 'Hybride']

In [40]:
# Encodage des variables catégorielles :

# Encodage de "Ft" :
df_clean_no_outliers_final = pd.get_dummies(df_clean_no_outliers_final, columns=['Ft'], prefix='Ft', drop_first=False)
bool_cols = df_clean_no_outliers_final.select_dtypes(include=['bool']).columns
df_clean_no_outliers_final[bool_cols] = df_clean_no_outliers_final[bool_cols].astype(int)

# Encodage de "Mk" : 

df_clean_no_outliers_final = pd.get_dummies(df_clean_no_outliers_final, columns=['Mk'], prefix='Mk', drop_first=False)
bool_cols = df_clean_no_outliers_final.select_dtypes(include=['bool']).columns
df_clean_no_outliers_final[bool_cols] = df_clean_no_outliers_final[bool_cols].astype(int)

In [41]:
# Vérifications finales du dataset après pré-traitement

display(df_clean_no_outliers_final.head())
print(df_clean_no_outliers_final.info())

,Cn,M (kg),Ewltp (g/km),Ec (cm3),Ep (KW),Erwltp (g/km),Year,Fc,Ft_Diesel,Ft_Essence,...,Mk_PEUGEOT,Mk_PORSCHE,Mk_RENAULT,Mk_SEAT,Mk_SKODA,Mk_SUBARU,Mk_SUZUKI,Mk_TOYOTA,Mk_VOLKSWAGEN,Mk_VOLVO
0,GIULIA,1504,178,1995,148,1.2,2021,7.9,0,1,...,0,0,0,0,0,0,0,0,0,0
1,GIULIA,1504,179,1995,148,1.2,2021,7.9,0,1,...,0,0,0,0,0,0,0,0,0,0
2,GIULIA,1504,180,1995,148,1.2,2021,8.0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,GIULIA,1520,159,1995,148,0.8,2021,7.0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,GIULIA,1520,160,1995,148,0.8,2021,7.0,0,1,...,0,0,0,0,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
Index: 173700 entries, 0 to 177493
Data columns (total 49 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Cn                  173700 non-null  object 
 1   M (kg)              173700 non-null  int64  
 2   Ewltp (g/km)        173700 non-null  int64  
 3   Ec (cm3)            173700 non-null  int64  
 4   Ep (KW)             173700 non-null  int64  
 5   Erwltp (g/km)       173700 non-null  float64
 6   Year                173700 non-null  int64  
 7   Fc                  173700 non-null  float64
 8   Ft_Diesel           173700 non-null  int64  
 9   Ft_Essence          173700 non-null  int64  
 10  Mk_ALFA ROMEO       173700 non-null  int64  
 11  Mk_ALLIED VEHICLES  173700 non-null  int64  
 12  Mk_ALPINE           173700 non-null  int64  
 13  Mk_AUDI             173700 non-null  int64  
 14  Mk_BENTLEY          173700 non-null  int64  
 15  Mk_BMW              173700 non-null  in

In [42]:
# Enregistrement du fichier de données prétraitées : 
# Définir les chemins vers les dossiers existants :
processed_dir = config.PROCESSED_DIR
metadata_dir = config.METADATA_DIR

# Créer les dossiers s'ils n'existent pas :
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(metadata_dir, exist_ok=True)

# Génération d'un timestamp au format YYYYMMDD_HHMMSS :
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f"DF_Processed_{timestamp}.csv"

# Construction du chemin complet vers le fichier dans le dossier raw existant : 
output_filepath = os.path.join(processed_dir, output_filename)

# Enregistrement du DataFrame dans le fichier avec le nom dynamique : 
df_clean_no_outliers_final.to_csv(output_filepath, index=False)

# Définition du chemin complet vers le fichier de métadonnées :
metadata_file = os.path.join(metadata_dir, "metadata.json")

# Chargement du contenu existant du fichier metadata, s'il existe, sinon initialisation d'un dictionnaire vide : 
if os.path.exists(metadata_file):
    with open(metadata_file, "r") as f:
        metadata = json.load(f)
else:
    metadata = {}

# Ajout ou mise à jour de la clé pour les données prétraitées : 
metadata["processed_data"] = output_filepath

# Réécriture du fichier metadata avec les deux informations (pour ne pas écraser celles des données brutes) : 
with open(metadata_file, "w") as f:
    json.dump(metadata, f, indent=4)